In [1]:
import pandas as pd
import numpy as np

In [22]:
train = pd.read_csv('../../dataset/train.csv')
test = pd.read_csv('../../dataset/test.csv')
# 前処理を一度にやるためにtrainとtestをconcatする
test['Transported'] = np.nan
train_test = pd.concat([train, test], axis=0, ignore_index=True, sort=False)

# split on `/` to cols (deck/num/side)
def split_cabin(df):
    cabin = df['Cabin'].str.split('/', expand=True).rename(columns={0: 'CabinDeck', 1: 'CabinNum', 2: 'CabinSide'})
    cabin['CabinNum'] = cabin['CabinNum'].astype(float)
    return pd.concat([df, cabin], axis=1)

train_test = split_cabin(train_test)

/var/folders/zp/6qwnpvfn0cs2whczwk_5pvqh0000gs/T/ipykernel_14638/933430717.py:5: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  train_test = pd.concat([train, test], axis=0, ignore_index=True, sort=False)


### 使う特徴量を選ぶ

In [23]:
train_test = train_test[['HomePlanet', 'CryoSleep', 'Destination', 'Age', 'CabinSide', 'VIP', 'Transported']]

採用する特徴量
- HomePlanet→最頻値
- CryoSleep→最頻値
- Destination→最頻値
- Age→中央値で埋める
    - 20歳以上以下でbin化する
- VIP→VIPなしで埋める
- CabinSide→CabinNum==82となっているCabinSideの最頻値で埋める？→Pで埋める

In [24]:
# 欠損値埋め
def fillna_cols(df):
    df['HomePlanet'] = df['HomePlanet'].fillna(df['HomePlanet'].mode()[0])
    df['CryoSleep'] = df['CryoSleep'].fillna(df['CryoSleep'].mode()[0])
    df['Destination'] = df['Destination'].fillna(df['Destination'].mode()[0])
    df['Age'] = df['Age'].fillna(df['Age'].median())
    df['VIP'] = df['VIP'].fillna(False)
    df['CabinSide'] = df['CabinSide'].fillna('P')
    return df

train_test = fillna_cols(train_test)

In [25]:
train_test.isna().sum(axis=0)

HomePlanet        0
CryoSleep         0
Destination       0
Age               0
CabinSide         0
VIP               0
Transported    4277
dtype: int64

#### 成人フラグを追加する

In [26]:
train_test['IsAdult'] = (train_test.Age >= 20).astype(int)
train_test.drop('Age', axis=1, inplace=True)

### Encoding

In [28]:
# HomePlanet, Destination, CabinSideはOne-hot化
train_test = pd.get_dummies(train_test, columns=['HomePlanet', 'Destination', 'CabinSide'], drop_first=True)

In [29]:
# boolをintへ
def bool2int(df):
    for col in df.columns:
        if df[col].dtype == bool:
            df[col] = df[col].astype(int)
    return df

train_test = bool2int(train_test)

In [30]:
train_test

,CryoSleep,VIP,Transported,IsAdult,HomePlanet_Europa,HomePlanet_Mars,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,CabinSide_S
0,0,0,0.0,1,1,0,0,1,0
1,0,0,1.0,1,0,0,0,1,1
2,0,1,0.0,1,1,0,0,1,1
3,0,0,0.0,1,1,0,0,1,1
4,0,0,1.0,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...
12965,1,0,NaN,1,0,0,0,1,1
12966,0,0,NaN,1,0,0,0,1,0
12967,1,0,NaN,1,0,1,0,0,0
12968,0,0,NaN,1,1,0,0,1,0


### モデリング

In [31]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [41]:
class MLP(nn.Module):
    def __init__(self, 
                 in_feats, 
                 out_feats, 
                 hid_feats=300, 
                 lr=0.01,
                 ):
        super().__init__()
        self.classifier = nn.Sequential(
            nn.Linear(in_feats, hid_feats),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(hid_feats, hid_feats),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(hid_feats, out_feats),
            nn.Dropout(0.2),
        )
        self.criterion = nn.BCEWithLogitsLoss()
        self.optimizer = torch.optim.RAdam(self.classifier.parameters(), lr=lr)
        
    def forward(self, x):
        x = self.classifier(x)
        return x
    
    def training_step(self, batch):
        self.train()
        x, y = batch
        self.optimizer.zero_grad()
        pr = self.forward(x)
        loss = self.criterion(pr, y)
        loss.backward()
        self.optimizer.step()
        return {'loss': loss, 'pred': F.sigmoid(pr)}
    
    def validation_step(self, batch):
        self.eval()
        x, y = batch
        with torch.no_grad():
            pr = self(x)
            loss = self.criterion(pr, y)
        return {'loss': loss, 'pred': F.sigmoid(pr)}
    
    def predict(self, x):
        self.eval()
        with torch.no_grad():
            pr = self(x)
        return F.sigmoid(pr)

### 学習

In [42]:
import os
import random
from sklearn.model_selection import KFold
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import Subset

In [43]:
def set_seed(seed=3407):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

SEED = 3407
set_seed(SEED)

In [44]:
# DataFrameをnp.ndarrayに変換
train = train_test[~train_test['Transported'].isna()]
test = train_test[train_test['Transported'].isna()]
# inputとlabelに分離
x_train = train.drop('Transported', axis=1).values
y_train = train.Transported.values[:, np.newaxis]
x_test = test.drop('Transported', axis=1).values

# Tensorにする
x_train = torch.tensor(x_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
x_test = torch.tensor(x_test, dtype=torch.float32)

# Datasetにまとめる
train_set = torch.utils.data.TensorDataset(x_train, y_train)

In [45]:
x_train.shape, x_test.shape, y_train.shape

(torch.Size([8693, 8]), torch.Size([4277, 8]), torch.Size([8693, 1]))

In [46]:
# hyperparameter
batch_size = 128
lr = 0.001
epochs = 200
num_hidden = 10

In [47]:
kf = KFold(n_splits=5, shuffle=True, random_state=SEED)

logs = {
    f'cv{i}': {
        'loss': [],
        'val_loss': [],
        'acc': [],
        'val_acc': [],
    }
    for i in range(1, 6)
}
logs['cv_acc'] = 0
logs['cv_loss'] = 0
device = 'cuda' if torch.cuda.is_available() else 'cpu'

for _fold, (train_index, valid_index) in enumerate(kf.split(train_set), 1):
    print(f'---------- cv{_fold} ------------')
    model = MLP(x_train.shape[1], 1, num_hidden, lr=lr).to(device)
    
    train_dataset = Subset(train_set, train_index)
    train_loader = DataLoader(train_dataset, batch_size, shuffle=True)
    valid_dataset = Subset(train_set, valid_index)
    valid_loader = DataLoader(valid_dataset, batch_size, shuffle=False)

    for epoch in range(epochs):
        train_loss = 0
        train_acc = 0
        val_loss = 0
        val_acc = 0

        # train
        for i, (X, y) in enumerate(train_loader):
            X, y = X.to(device), y.to(device)
            outputs = model.training_step((X, y))
            train_loss += outputs['loss'].item()
            train_acc += torch.sum((outputs['pred'] >= 0.5) * (y == 1), axis=0).item()
        train_loss = train_loss / i  # batchごとにlossがmeanされてるため、iteration数で割ることで全batchの平均lossになる
        train_acc = train_acc / len(train_loader.dataset)

        # val
        for X, y in valid_loader:
            X, y = X.to(device), y.to(device)
            outputs = model.validation_step((X, y))
            val_loss += outputs['loss'].item()
            val_acc += torch.sum((outputs['pred'] >= 0.5) * (y == 1), axis=0).item()
        val_loss = val_loss / i
        val_acc = val_acc / len(valid_loader.dataset)

        print (f'Epoch [{epoch+1}/{epochs}], loss: {train_loss:.4f}, val_loss: {val_loss:.4f},  acc: {train_acc:.4f}, val_acc: {val_acc:.4f}')
        logs[f'cv{_fold}']['loss'].append(train_loss)
        logs[f'cv{_fold}']['acc'].append(train_acc)
        logs[f'cv{_fold}']['val_loss'].append(val_loss)
        logs[f'cv{_fold}']['val_acc'].append(val_acc)

    logs[f'cv{_fold}']['model'] = model
    logs['cv_loss'] += val_loss / kf.n_splits
    logs['cv_acc'] += val_acc / kf.n_splits

print('CV loss', logs['cv_loss'], 'CV acc', logs['cv_acc'])

---------- cv1 ------------
Epoch [1/200], loss: 0.7115, val_loss: 0.1806,  acc: 0.0985, val_acc: 0.0000


/Users/yamada/.pyenv/versions/3.8.0/lib/python3.8/site-packages/torch/nn/functional.py:1944: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch [2/200], loss: 0.7087, val_loss: 0.1798,  acc: 0.0975, val_acc: 0.0000
Epoch [3/200], loss: 0.7051, val_loss: 0.1789,  acc: 0.1043, val_acc: 0.0000
Epoch [4/200], loss: 0.7004, val_loss: 0.1774,  acc: 0.1159, val_acc: 0.0339
Epoch [5/200], loss: 0.6938, val_loss: 0.1751,  acc: 0.2087, val_acc: 0.2340
Epoch [6/200], loss: 0.6827, val_loss: 0.1708,  acc: 0.3204, val_acc: 0.3220
Epoch [7/200], loss: 0.6657, val_loss: 0.1654,  acc: 0.3571, val_acc: 0.3306
Epoch [8/200], loss: 0.6471, val_loss: 0.1599,  acc: 0.3693, val_acc: 0.3399
Epoch [9/200], loss: 0.6317, val_loss: 0.1548,  acc: 0.3664, val_acc: 0.3393
Epoch [10/200], loss: 0.6222, val_loss: 0.1506,  acc: 0.3540, val_acc: 0.3174
Epoch [11/200], loss: 0.6146, val_loss: 0.1478,  acc: 0.3585, val_acc: 0.3209
Epoch [12/200], loss: 0.6098, val_loss: 0.1460,  acc: 0.3556, val_acc: 0.3122
Epoch [13/200], loss: 0.6111, val_loss: 0.1452,  acc: 0.3582, val_acc: 0.3122
Epoch [14/200], loss: 0.6030, val_loss: 0.1446,  acc: 0.3527, val_acc: 0